<a href="https://colab.research.google.com/github/aetev/Learning-stuff-/blob/main/trying%20somthing%20new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import yfinance as yfi
import pandas as pd
import asyncio

In [39]:
def proccess_data(df):
    # Ensure the input is a DataFrame
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")

    proccessed_data = pd.DataFrame()

    # Calculate the range
    proccessed_data['range'] = df['High'] - df['Low']

    stdev = proccessed_data['range'].std()

    # Calculate the percentage of the range that the closing price represents
    proccessed_data['close_percentage'] = (df['Close'] - df['Low']) / proccessed_data['range']

    proccessed_data['range'] = proccessed_data['range'] / stdev

    # Calculate the difference between the closing and opening prices
    proccessed_data['price_difference'] = (df['Close'] - df['Open']) / stdev

    return proccessed_data

In [69]:
async def fetch_ticker_data(ticker: str, start_date: str, end_date: str, interval: str):
    try:
        data = yfi.download(ticker, start=start_date, end=end_date, interval=interval)
        proccessed_data = proccess_data(data)
        return proccessed_data

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

async def fetch_all_tickers(ticker_list: list,start_date: str,end_date: str, interval: str, max_concurrency:int=50):
    tasks = [fetch_ticker_data(ticker,start_date=start_date,end_date=end_date,interval=interval) for ticker in ticker_list]
    return await asyncio.gather(*tasks[:max_concurrency])

In [78]:
start_date = '2023-01-01'
end_date = '2023-10-22'
interval='60m'

# List of forex tickers    # List of forex tickers
forex_tickers = ["EURUSD=X", "GBPUSD=X", "USDJPY=X", "AUDUSD=X"]  # Example tickers

# Fetch data concurrently
results = await fetch_all_tickers(forex_tickers,start_date=start_date,end_date=end_date,interval=interval)

combined_df = pd.concat(results, axis=0, ignore_index=True)
combined_df.dropna(inplace=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [82]:
target = combined_df['price_difference']
print(target)
print(target[1:])

0        0.868053
1       -0.372299
2        0.000000
3        0.372170
4       -0.743954
           ...   
19952    0.028981
19953   -0.468389
19954   -0.212170
19955    0.429098
19956   -0.164084
Name: price_difference, Length: 19838, dtype: float64
1       -0.372299
2        0.000000
3        0.372170
4       -0.743954
5       -0.494980
           ...   
19952    0.028981
19953   -0.468389
19954   -0.212170
19955    0.429098
19956   -0.164084
Name: price_difference, Length: 19837, dtype: float64


In [83]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from sklearn.model_selection import train_test_split

model = Sequential([
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

huber_loss = Huber(delta=1.35)
model.compile(optimizer=Adam(), loss=huber_loss)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)